In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
#basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"

In [3]:
#akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"

In [4]:
#ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

In [5]:
#basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [6]:
#akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [7]:
#ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

In [2]:
'''import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")'''

'import os\nos.makedirs(\'Data/\',exist_ok=True) \n# Confirm folder created\nos.listdir("Data/")'

In [9]:
## Save current dataframe to file.
#basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [10]:
## Save current dataframe to file.
#akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [11]:
## Save current dataframe to file.
#ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [3]:
# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [4]:
# Open saved file and preview again
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0
1,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
2,tt0000005,10,Blacksmith Scene,US,\N,imdbDisplay,\N,0
3,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
4,tt0000005,6,Blacksmith Scene #1,US,\N,alternative,\N,0


In [5]:
# Open saved file and preview again
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1922
1,tt0000002,5.8,259
2,tt0000003,6.5,1734
3,tt0000004,5.6,174
4,tt0000005,6.2,2546


In [6]:
basics.isna().sum()

tconst                0
titleType             0
primaryTitle          0
originalTitle         0
isAdult               0
startYear             0
endYear           83435
runtimeMinutes        0
genres                0
dtype: int64

In [7]:
#Replace "\N" with np.nan
basics.replace({'\\N':np.nan},inplace=True)

In [8]:
basics.isna().sum()

tconst                0
titleType             0
primaryTitle          0
originalTitle         0
isAdult               0
startYear             0
endYear           83435
runtimeMinutes        0
genres                0
dtype: int64

In [9]:
basics['runtimeMinutes'].value_counts(dropna=False)

90     5758
95     2513
85     2471
80     2444
100    2427
       ... 
397       1
675       1
409       1
375       1
285       1
Name: runtimeMinutes, Length: 335, dtype: int64

In [10]:
#Eliminate movies that are null for runtimeMinutes
basics.dropna(subset=['runtimeMinutes'], inplace=True)


In [11]:
#Eliminate movies that are null for genre and startYear
basics.dropna(subset=['startYear','genres'], inplace=True)


In [12]:
basics['startYear'] = basics['startYear'].astype(int)

In [13]:
basics.isna().sum()

tconst                0
titleType             0
primaryTitle          0
originalTitle         0
isAdult               0
startYear             0
endYear           83435
runtimeMinutes        0
genres                0
dtype: int64

In [15]:
#keep only titleType==Movie
basics_movie = basics['titleType']=='movie'
basics=basics.loc[basics_movie]

In [16]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [17]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83435 entries, 0 to 83434
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          83435 non-null  object 
 1   titleType       83435 non-null  object 
 2   primaryTitle    83435 non-null  object 
 3   originalTitle   83435 non-null  object 
 4   isAdult         83435 non-null  int64  
 5   startYear       83435 non-null  int32  
 6   endYear         0 non-null      float64
 7   runtimeMinutes  83435 non-null  int64  
 8   genres          83435 non-null  object 
dtypes: float64(1), int32(1), int64(2), object(5)
memory usage: 6.0+ MB


In [18]:
#keep startYear 2000-2022
Y2000_22= basics[(basics['startYear']>=2000) & (basics['startYear']<=2022)]

#basics=basics.loc[Y2000_22]


In [19]:
basics= Y2000_22

In [20]:
#Eliminate movies that include "Documentary" in genre 
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

In [21]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0
1,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
2,tt0000005,10,Blacksmith Scene,US,\N,imdbDisplay,\N,0
3,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
4,tt0000005,6,Blacksmith Scene #1,US,\N,alternative,\N,0


In [22]:
akas.replace({'\\N':np.nan})

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0
...,...,...,...,...,...,...,...,...
1366977,tt9916702,1,Loving London: The Playground,US,NaN,NaN,NaN,0
1366978,tt9916720,10,The Demonic Nun,US,NaN,tv,NaN,0
1366979,tt9916720,12,The Nun 2,US,NaN,imdbDisplay,NaN,0
1366980,tt9916756,1,Pretty Pretty Black Girl,US,NaN,imdbDisplay,NaN,0


In [23]:
#Eliminate region that are null 
akas.dropna(subset=['region'], inplace=True)

In [24]:
US_Only = akas['region'].str.contains('US',case=False)
akas = akas[US_Only]

In [25]:
#Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)
keepers =basics['tconst'].isin(akas['titleId'])
keepers

0        True
1        True
2        True
3        True
4        True
         ... 
83430    True
83431    True
83432    True
83433    True
83434    True
Name: tconst, Length: 83435, dtype: bool

In [26]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...
83430,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
83431,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
83432,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
83433,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


In [27]:
basics.to_csv('basics.csv')

In [28]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1922
1,tt0000002,5.8,259
2,tt0000003,6.5,1734
3,tt0000004,5.6,174
4,tt0000005,6.2,2546


In [29]:
#Replace "\N" with np.nan (if any)
ratings.replace({'\\N':np.nan})

,tconst,averageRating,numVotes
0,tt0000001,5.7,1922
1,tt0000002,5.8,259
2,tt0000003,6.5,1734
3,tt0000004,5.6,174
4,tt0000005,6.2,2546
...,...,...,...
1246999,tt9916690,7.4,6
1247000,tt9916720,5.3,275
1247001,tt9916730,9.2,7
1247002,tt9916766,6.7,21


In [30]:
#Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)
Ratings_US =ratings['tconst'].isin(akas['titleId'])
Ratings_US


0           True
1           True
2          False
3          False
4           True
           ...  
1246999    False
1247000     True
1247001    False
1247002    False
1247003    False
Name: tconst, Length: 1247004, dtype: bool

In [31]:
ratings = ratings[Ratings_US]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1922
1,tt0000002,5.8,259
4,tt0000005,6.2,2546
5,tt0000006,5.1,175
6,tt0000007,5.4,797
...,...,...,...
1246978,tt9916204,8.2,247
1246984,tt9916348,8.5,17
1246985,tt9916362,6.4,4985
1246989,tt9916428,3.8,14


In [32]:
ratings.to_csv('ratings.csv')

In [33]:
akas.to_csv('akas.csv')